In [1]:
%pip -q install google-genai

In [51]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [50]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [52]:
!pip install -q google-adk

In [49]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [48]:
def call_agent(agent: Agent, message_text: str) -> str:

    session_service = InMemorySessionService()

    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [47]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [44]:
def agente_buscador(tema_anime, data_atual):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de pesquisa focado em vasculhar a internet utilizando a ferramenta (google_search) para encontrar os 10 melhores animes sobre o tema apontado pelo usuario, os pontos que voce ira considerar são:
        -ranques em sites renomados do assunto
        -numeros de vizualizaçoes
        -opinioes em redes sociais
        -inovação
        É importante ressaltar que as opinioes nas redes sociais precisam ser atuais, ou tambem bastante engajadoras se possivel.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_buscador = f"tema: {tema_anime}\nData de hoje: {data_atual}"

    animes_encontrados = call_agent(buscador, entrada_buscador)
    return animes_encontrados

In [43]:
def agente_planejador(tema_anime, animes_encontrados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction=""" Voce é um agente focado em listar os animes pesquisados pelo outro agente, voce deve focar em ser sucinto mas mantes todas as informaçoes importantes, seja tambem natural em seus resultados,
         alem de apontar suas datas de lançamento, você pode ranquea-los tambem com base em notas dadas na internet utilize a ferramenta (google_search) para isso""",
        description="Agente planejador de lista animes",
        tools=[google_search]
    )

    entrada_planejador = f"tema:{tema_anime}\nAnimes buscados: {animes_encontrados}"

    lista_animes_planejada = call_agent(planejador, entrada_planejador)
    return lista_animes_planejada

In [42]:
def agente_redator(tema_anime, lista_animes_planejada):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Voce ira redigir um post de top 10, ou seja você utilizara a lista fornecida pelo outro agente para preparar um post para o tiktok, com legenda e conteudo do post descrito. Utilize uma linguajem focada no assunto,
            se quiser referencie animes conhecidos em seus textos.
            """,
        description="Agente redator de posts engajadores para o TikTok"
    )
    entrada_redator = f"tema: {tema_anime}\nPlano de post: {lista_animes_planejada}"

    rascunho_post = call_agent(redator, entrada_redator)
    return rascunho_post

In [41]:
def agente_revisor(tema_anime, rascunho_post):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no TikTok.
            Os posts que você vai revisar são focados na faixa etaria de 10 a 30 anos.
            A pagina utiliza um linguagem mais natural, mas noa são aceitos erros de gramatica
            Você revisará também os fatos descritos nos posts, se eles realmente são veridicos, inclusive os referenciando os proprios animes
            Se não houver problemas com o post, apenas diga "O seu post ficou massa em",
            caso haja problemas os aponte e corrija-os
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_revisor = f"tema: {tema_anime}\nRascunho: {rascunho_post}"
    # Executa o agente
    post_revisado = call_agent(revisor, entrada_revisor)
    return post_revisado

In [58]:
data_atual = date.today().strftime("%d/%m/%Y")
print("Olá sou o criador de posts sobre animes mais conhecido no mundo ninja")

# --- Obter o tema do Usuário ---
tema_anime = input("Por favor digite seu tema que tem interesse: ")

if not tema_anime:
    print("Você esqueceu de digitar o seu tema!")
else:
    print(f"Ótimo, vamos encontrar os melhores animes para você 🫵, e ainda bolar um post para bombar no seu TikTok 💥")

    animes_encontrados = agente_buscador(tema_anime, data_atual)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(animes_encontrados))
    print("--------------------------------------------------------------")

    lista_animes_planejada = agente_planejador(tema_anime, animes_encontrados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(lista_animes_planejada))
    print("--------------------------------------------------------------")

    rascunho_post = agente_redator(tema_anime, lista_animes_planejada)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(tema_anime, rascunho_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

Olá sou o criador de posts sobre animes mais conhecido no mundo ninja
Por favor digite seu tema que tem interesse: porradaria
Ótimo, vamos encontrar os melhores animes para você 🫵, e ainda bolar um post para bombar no seu TikTok 💥

--- 📝 Resultado do Agente 1 (Buscador) ---



> Okay, aqui estão minhas buscas para encontrar os 10 melhores animes de "porradaria", considerando rankings, visualizações, opiniões recentes/engajadoras em redes sociais e inovação:
> 
> 
> Com base nas minhas buscas, aqui estão 10 animes de "porradaria" que se destacam em 2025:
> 
> 1.  **Baki** : Conhecido por suas cenas de luta brutais e detalhadas artes marciais. A história segue Baki Hanma em sua busca para superar seu pai, que é considerado o ser humano mais forte do mundo.
> 
> 2.  **Kenichi: O Discípulo Mais Forte da História**: Acompanha Kenichi Shirahama, um estudante que decide aprender artes marciais para se defender de valentões.
> 
> 3.  **One-Punch Man**: Apresenta Saitama, um herói que derrota qualquer inimigo com um único soco, subvertendo as expectativas com lutas insanas. A animação dinâmica e os confrontos são visualmente impressionantes. A terceira temporada é muito aguardada.
> 
> 4.  **My Hero Academia**: As batalhas impressionam pela criatividade no uso de habilidades e coreografias dinâmicas. Acompanha Izuku Midoriya, um jovem sem poderes que herda o lendário One For All. A série está chegando à sua temporada final em 2025.
> 
> 5.  **Demon Slayer: Kimetsu no Yaiba**: Visualmente impressionante, com animação de alta qualidade e lutas de espadas espetaculares. Tanjiro Kamado enfrenta demônios poderosos em busca de vingança e para salvar sua irmã.
> 
> 6.  **Dragon Ball Z**: Um clássico que popularizou as lutas prolongadas e cheias de energia. As transformações de Goku e Vegeta resultam em batalhas intensas. A franquia continua sendo uma referência no gênero.
> 
> 7.  **Jujutsu Kaisen**: Apresenta um mundo onde todos têm energia amaldiçoada, e feiticeiros Jujutsu lutam contra maldições. As lutas são intensas e o enredo é envolvente.
> 
> 8.  **Record of Ragnarok**: Um torneio onde deuses de diversas mitologias enfrentam os guerreiros mais poderosos da história humana. Cada batalha é um show de força e estratégia.
> 
> 9.  **Megalo Box**: Uma abordagem futurista do boxe, onde os lutadores usam exoesqueletos para amplificar suas habilidades. Junk Dog compete no maior torneio sem usar tecnologia.
> 
> 10. **Solo Leveling**: Um anime muito popular, com a segunda temporada sendo muito aguardada. A história segue Sung Jinwoo, um caçador de monstros que se torna incrivelmente poderoso.
> 
> Esses animes combinam ação intensa, personagens cativantes e histórias envolventes, tornando-os imperdíveis para fãs de "porradaria".


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, aqui está uma lista concisa dos 10 melhores animes de "porradaria" para você, com foco nas datas de lançamento e suas classificações relevantes:
> 
> 1.  **Baki**: Lançado em 2018, conhecido por suas lutas brutais.
> 
> 2.  **Kenichi: O Discípulo Mais Forte da História**: Estreou em 2006, ideal para quem gosta de artes marciais.
> 
> 3.  **One-Punch Man**: Lançado em 2015, famoso por subverter o gênero com lutas insanas. Nota: geralmente acima de 8/10 em sites de avaliação.
> 
> 4.  **My Hero Academia**: Desde 2016, com batalhas criativas e dinâmicas. Nota: Similarmente bem avaliado, com notas acima de 8/10.
> 
> 5.  **Demon Slayer: Kimetsu no Yaiba**: Lançado em 2019, visualmente impressionante. Nota: frequentemente acima de 8.5/10 devido à sua animação.
> 
> 6.  **Dragon Ball Z**: Um clássico dos anos 80 e 90, referência no gênero.
> 
> 7.  **Jujutsu Kaisen**: Desde 2020, com lutas intensas e enredo envolvente. Nota: Costuma ter notas elevadas, acima de 8/10.
> 
> 8.  **Record of Ragnarok**: Lançado em 2021, com batalhas mitológicas épicas.
> 
> 9.  **Megalo Box**: Estreou em 2018, uma abordagem futurista do boxe.
> 
> 10. **Solo Leveling**: Lançado em 2024, rapidamente se tornou popular.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🔥 **TOP 10 ANIMAÇÕES DE PORRADARIA QUE VÃO TE FAZER QUEBRAR O SOFÁ!** 🔥
> 
> Fala, seus viciados em adrenalina! Se você é do tipo que curte uma boa luta, prepare-se, porque o vídeo de hoje vai te deixar no hype! 😎
> 
> **🎬 Conteúdo do vídeo:**
> 
> *   Cenas INSANAS de cada anime, mostrando a pancadaria rolando solta.
> *   Edição dinâmica com transições rápidas e efeitos impactantes.
> *   Música ÉPICA de fundo que combine com a vibe de cada luta.
> *   Apresentação rápida e ENGRAÇADA de cada anime, com legendas chamativas.
> *   Enquete no final: Qual desses animes tem a luta mais FODA?
> 
> **📝 Legenda:**
> 
> "Se liga no TOP 10 de animes de porradaria que vão te deixar sem fôlego! 💥 De clássicos como Dragon Ball Z até os lançamentos mais INSANOS como Solo Leveling, tem luta pra todo gosto! Já prepara o energético e vem comigo! 🔥
> 
> 1️⃣ **Baki** (2018): Brutalidade PURA!
> 2️⃣ **Kenichi** (2006): Pra quem AMA artes marciais! 🥋
> 3️⃣ **One-Punch Man** (2015): Um soco pra resolver TUDO! 👊
> 4️⃣ **My Hero Academia** (2016): Lutando pra virar o número 1! 🥇
> 5️⃣ **Demon Slayer** (2019): Visual que CORTA a respiração! 👹
> 6️⃣ **Dragon Ball Z** (Anos 80/90): O clássico que a gente AMA! 🐉
> 7️⃣ **Jujutsu Kaisen** (2020): Prepare-se para a MALDIÇÃO! 😈
> 8️⃣ **Record of Ragnarok** (2021): Deuses e humanos NA PORRADA! ⚔️
> 9️⃣ **Megalo Box** (2018): Boxe futurista que DÁ CHOQUE! 🤖
> 🔟 **Solo Leveling** (2024): O novato que já CHEGOU COM TUDO! 👑
> 
> Qual desses é o seu PREFERIDO? Deixa nos comentários! 👇 #anime #porradaria #luta #recomendações #otaku #animes"


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O seu post ficou massa em, so faria alguns ajustes:
> 
> 1.  **Baki (2018):** Em vez de "Brutalidade PURA!", que tal algo como "Prepare-se para quebrar uns ossos!"? É mais chamativo e mantém a vibe.
> 2.  **Kenichi (2006):** Trocar "Pra quem AMA artes marciais!" por "Aqui a lapada come solta!".
> 3.  **My Hero Academia (2016):** Substituir "Lutando pra virar o número 1!" por "Lutando para se tornar o Herói N° 1!".
> 4.  **Dragon Ball Z (Anos 80/90):** Trocar "O clássico que a gente AMA!" por "O clássico que ensinou a carregar energia!".
> 5.  **Solo Leveling (2024):** Trocar "O novato que já CHEGOU COM TUDO!" por "O novato que chegou dando porrada e virou rei!".
> 6.  **Hashtags:** Adicionar hashtags como #animesbrasil e #mundootaku para aumentar o alcance.


--------------------------------------------------------------
